# LSTM
** 长短期记忆long short-term memory **:  
遗忘门:控制上一时间步的记忆细胞 
输入门:控制当前时间步的输入  
输出门:控制从记忆细胞到隐藏状态  
记忆细胞：⼀种特殊的隐藏状态的信息的流动  


![Image Name](https://cdn.kesci.com/upload/image/q5jk2bnnej.png?imageView2/0/w/640/h/640)

$$
I_t = σ(X_tW_{xi} + H_{t−1}W_{hi} + b_i) \\
F_t = σ(X_tW_{xf} + H_{t−1}W_{hf} + b_f)\\
O_t = σ(X_tW_{xo} + H_{t−1}W_{ho} + b_o)\\
\widetilde{C}_t = tanh(X_tW_{xc} + H_{t−1}W_{hc} + b_c)\\
C_t = F_t ⊙C_{t−1} + I_t ⊙\widetilde{C}_t\\
H_t = O_t⊙tanh(C_t)
$$


In [1]:
import torch 
from torch import nn
import numpy as np
import time 
import math
import sys

### 工具函数

In [2]:
def load_data_jay_lyrics():
    with open('../Datasets/jaychou_lyrics.txt') as f:
        corpus_chars = f.read()
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
#     corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

corpus_indices, char_to_idx, idx_to_char, vocab_size = load_data_jay_lyrics()

In [3]:
def one_hot(x, n_class, dtype=torch.float32):
    result = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)  # shape: (n, n_class)
    result.scatter_(1, x.long().view(-1, 1), 1)  # result[i, x[i, 0]] = 1
    return result

def to_onehot(X, n_class):
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

In [4]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices = torch.tensor(corpus_indices, dtype=torch.float32, device=device)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0: batch_size*batch_len].view(batch_size, batch_len)
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [5]:
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta/norm)

In [6]:
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size*(2 if rnn_layer.bidirectional else 1)
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)
    def forward(self, inputs, state):
        # inputs shape: [batch_size, num_steps]
        X = to_onehot(inputs, vocab_size)
        X = torch.stack(X) #X.shape: [num_steps, batch_size, vocab_size]
        hiddens, state = self.rnn(X,state)
        hiddens = hiddens.view(-1, hiddens.shape[-1])
        output = self.dense(hiddens)
        return output, state

In [7]:
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char, char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor(output[-1],device=device).view(-1,1)
        (Y, state) = model(X,state)
        if t < len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(Y.argmax(dim=1).item())
    return ''.join([idx_to_char[i] for i in output ])
        

In [8]:
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        state = None
        for X, Y in data_iter:
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态
                if isinstance (state, tuple): # LSTM, state:(h, c)  
                    state[0].detach_()
                    state[1].detach_()
                else: 
                    state.detach_()
            (output, state) = model(X, state) # output.shape: (num_steps * batch_size, vocab_size)
            y = torch.flatten(torch.transpose(Y,0,1))
            l = loss(output, y.long())
            
            optimizer.zero_grad()
            l.backward()
            grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

### LSTM训练

In [9]:
device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')
num_hiddens = 256
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e-2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']
lstm_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens)
model = RNNModel(lstm_layer, vocab_size)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.048820, time 0.58 sec
 - 分开了叫痛 是不是说 没有做完的梦最痛 迷路的后果 我能承受 这最后的出口 在爱过了才有 送给最爱的人F
 - 不分开也知道不能没有孤寂 感激你让我拥有缺点的美丽 看着那白色的蜻蜓 在空中忘了前进 还能不能 重新编织 
epoch 80, perplexity 1.025490, time 0.78 sec
 - 分开了叫痛 是不是说 没有做完的梦最痛 迷路的后果 我能承受 这最后的出口 在爱过了才有 送给最爱的人F
 - 不分开 爱情来的太快就像龙卷风 离不开暴风圈来不及逃 我不能再想 我不能再想 我不 我不 我不能 爱情走的
epoch 120, perplexity 1.045585, time 0.57 sec
 - 分开了叫痛 是不是说 没有做完的梦最痛 迷路的后果 我能承受 这最后一场白 青春是干净的纯白 像一遍绿地
 - 不分开 总能不能原谅我 请不要把分手当作你的请求 我知道坚持要走是你受伤的借口 请你回头 我会陪你一直走到
epoch 160, perplexity 1.222533, time 0.82 sec
 - 分开 叫梦的画面 残忍地温柔出现 脆弱时间到 我们一起来祷告 仁慈的父我已坠入 看不见罪的国度 请原谅我
 - 不分开 为什么还要我用微笑来带过 我没有紧紧抱住你  你紧紧抱住你 不会有三块 我是不是 啊  感觉到 也


# GRU

RNN存在的问题：梯度较容易出现衰减或爆炸（BPTT）  
⻔控循环神经⽹络：捕捉时间序列中时间步距离较⼤的依赖关系  
**RNN**:  


![Image Name](https://cdn.kesci.com/upload/image/q5jjvcykud.png?imageView2/0/w/320/h/320)


$$
H_{t} = ϕ(X_{t}W_{xh} + H_{t-1}W_{hh} + b_{h})
$$
**GRU**:


![Image Name](https://cdn.kesci.com/upload/image/q5jk0q9suq.png?imageView2/0/w/640/h/640)



$$
R_{t} = σ(X_tW_{xr} + H_{t−1}W_{hr} + b_r)\\    
Z_{t} = σ(X_tW_{xz} + H_{t−1}W_{hz} + b_z)\\  
\widetilde{H}_t = tanh(X_tW_{xh} + (R_t ⊙H_{t−1})W_{hh} + b_h)\\
H_t = Z_t⊙H_{t−1} + (1−Z_t)⊙\widetilde{H}_t
$$
• 重置⻔有助于捕捉时间序列⾥短期的依赖关系；  
• 更新⻔有助于捕捉时间序列⾥⻓期的依赖关系。    

In [10]:
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = RNNModel(gru_layer, vocab_size).to(device)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 5.341638, time 0.60 sec
 - 分开都迁就反手 你转身向 你脸上  我用Peace 我马儿有些风霜落寞 表哥人潮般内容 等到 一直走不灭
 - 不分开 差狠接剪 琴发 你用J餐 是谁 用古老的咒留在兵戎装过 过过过 过过过 过过过的刚才留的风沙 过去
epoch 80, perplexity 1.504238, time 0.81 sec
 - 分开的叫醒 是什么形状 是离开 口袋里学 梦在地面对我 我又多传了一下往的画面 被岁月覆盖 我的想活 把
 - 不分开 爱完血后开进 相同 我点燃烛火不想送 跌跌撞撞 我中间的人群走过 清晰透明的小舢板 画 我身世人生
epoch 120, perplexity 1.269553, time 0.56 sec
 - 分开的让我疯掉 我的烦恼 想爬失去年的清天 人爱是我要的说 家乡事不准说 我想我这首歌的嘴 不要再这样 
 - 不分开场 谁都伤不少我才孤单的想念你想念若想回 心情面临他们的手中只有一点 从小到 我假装过 心碎内心就这
epoch 160, perplexity 1.682628, time 0.82 sec
 - 分开进隧道风这样子 你的人们 开始在雕刻   我个人 不要闹我 咱姥姥烧柴煮水饺 放下枪 不同 我轻轻地
 - 不分开 对不该 因为无心啊  直说的名叫来 马蹄声 很可惜没有胆敢与我自己 人们心跳你很累 想要对面对我的


# 深度循环神经网络  

![Image Name](https://cdn.kesci.com/upload/image/q5jk3z1hvz.png?imageView2/0/w/320/h/320)


$$
\boldsymbol{H}_t^{(1)} = \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(1)} + \boldsymbol{H}_{t-1}^{(1)} \boldsymbol{W}_{hh}^{(1)} + \boldsymbol{b}_h^{(1)})\\
\boldsymbol{H}_t^{(\ell)} = \phi(\boldsymbol{H}_t^{(\ell-1)} \boldsymbol{W}_{xh}^{(\ell)} + \boldsymbol{H}_{t-1}^{(\ell)} \boldsymbol{W}_{hh}^{(\ell)} + \boldsymbol{b}_h^{(\ell)})\\
\boldsymbol{O}_t = \boldsymbol{H}_t^{(L)} \boldsymbol{W}_{hq} + \boldsymbol{b}_q
$$

In [11]:
gru_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens,num_layers=2)
model = RNNModel(gru_layer, vocab_size).to(device)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.048538, time 0.94 sec
 - 分开的阳光 每天有大多少就知道 开一瓶冲仔救止成个人 东吴远刀落的那爱女人 漂亮的让我面红的可爱女人 温
 - 不分开心永一力的方境 原来爱家个身笔 那么会扯去扯铃　 扯多你就会上瘾 扯你最善变的表情　 我的解释请你务
epoch 80, perplexity 1.022320, time 0.97 sec
 - 分开也对 你说为了装饰 请问干我啥事 是不是只能用相机纪录自然拿给下一代下一代回味 可怜可悲 森林绿地都
 - 不分开 妈妈的话 别让她受伤 想快快长大 才能保护她 美丽的白发 幸福中发芽 天使的魔法 温暖中慈祥 屋檐
epoch 120, perplexity 1.016725, time 0.97 sec
 - 分开也知道 你没有舍不得 你说你也会难过我不相信 牵着你陪着我 也只是曾经 希望他是真的比我还要爱你 我
 - 不分开 话说然一个你 不想太多 我想一定是我听错弄错搞错 拜托 我想是你的脑袋有问题 随便说说 其实我早已
epoch 160, perplexity 1.015204, time 1.09 sec
 - 分开又不会让你们竖起大姆指 生活不该有公式 我可以随性跳芭蕾舞 照节拍 手放开 静下来 像一只天鹅把脚尖
 - 不分开编力的很荒凉 落花配对配夕阳 翻山越岭渡过江 我清一清嗓 清一清嗓 唱起秦腔 飞天飞敦煌 北方北大荒


# 双向循环神经网络 

![Image Name](https://cdn.kesci.com/upload/image/q5j8hmgyrz.png?imageView2/0/w/320/h/320)

$$ 
\begin{aligned} \overrightarrow{\boldsymbol{H}}_t &= \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(f)} + \overrightarrow{\boldsymbol{H}}_{t-1} \boldsymbol{W}_{hh}^{(f)} + \boldsymbol{b}_h^{(f)})\\
\overleftarrow{\boldsymbol{H}}_t &= \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(b)} + \overleftarrow{\boldsymbol{H}}_{t+1} \boldsymbol{W}_{hh}^{(b)} + \boldsymbol{b}_h^{(b)}) \end{aligned} $$
$$
\boldsymbol{H}_t=(\overrightarrow{\boldsymbol{H}}_{t}, \overleftarrow{\boldsymbol{H}}_t)
$$
$$
\boldsymbol{O}_t = \boldsymbol{H}_t \boldsymbol{W}_{hq} + \boldsymbol{b}_q
$$

In [12]:
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens,bidirectional=True)
model = RNNModel(gru_layer, vocab_size).to(device)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.000068, time 1.00 sec
 - 分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开
 - 不分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开
epoch 80, perplexity 1.000014, time 0.98 sec
 - 分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开
 - 不分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开
epoch 120, perplexity 1.000004, time 0.95 sec
 - 分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开
 - 不分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开
epoch 160, perplexity 1.114192, time 1.02 sec
 - 分开读开始有时有些犹豫犹豫犹豫犹碰到碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰碰
 - 不分开心开心开心开心开心开心开心开心开心开心开心开心开心开心开心开心开心开心开心开心开心开心开心开心开心开
